# Đề thi mẫu chứng chỉ TensorFlow - Dạng bài 04

```
ProtonX - TensorFlow Class
```
Giới thiệu về chứng chỉ - https://www.tensorflow.org/certificate

> Chú ý: đọc kỹ các nội dung trong Handle Notebook trong liên kết trên.

---

**Bài toán: Phân loại câu mỉa mai !**

### Hướng dẫn làm bài  
- Trong phần bài tập này, các bạn sẽ thực hiện tương tự bài IMDB Dataset.

Yêu cầu:
- Sử dụng TensorFlow.
- Sử dụng Callback.
- Sử dụng các mạng LSTM, GRU, Bi-LSTM, v.v...
- Tỷ lệ chính xác trên tập Test > 85 - 87%.



### Thực hiện

Dataset: https://rishabhmisra.github.io/publications/

In [1]:
# Hướng dẫn về bộ dữ liệu này được thực hiện trong Lab Tokenizer
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/sarcasm.json \
    -O /tmp/sarcasm.json

import json

with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

--2024-02-09 04:04:49--  https://storage.googleapis.com/learning-datasets/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.195.207, 173.194.196.207, 173.194.197.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.195.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.04s   

2024-02-09 04:04:49 (121 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential

In [3]:
datastore

[{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
  'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
  'is_sarcastic': 0},
 {'article_link': 'https://www.huffingtonpost.com/entry/roseanne-revival-review_us_5ab3a497e4b054d118e04365',
  'headline': "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
  'is_sarcastic': 0},
 {'article_link': 'https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697',
  'headline': "mom starting to fear son's web series closest thing she will have to grandchild",
  'is_sarcastic': 1},
 {'article_link': 'https://politics.theonion.com/boehner-just-wants-wife-to-listen-not-come-up-with-alt-1819574302',
  'headline': 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
  'is_sarcastic': 1},
 {'article_link': 'https://www.huffingtonpost.com/entry/jk-rowling-w

In [7]:
import numpy as np

In [15]:
dataset = []
label_dataset = []

for item in datastore:
  dataset.append(item['headline'])
  label_dataset.append(item['is_sarcastic'])

dataset = np.array(dataset)
label_dataset = np.array(label_dataset)
train_size = 0.8
size = int(len(dataset)*train_size)

train_sentences = dataset[:size]
test_sentences = dataset[size:]

train_label = label_dataset[:size]
test_label = label_dataset[size:]


In [16]:
train_sentences.shape

(21367,)

In [17]:
vocab_size = len(train_sentences)
embedding_size = 64
max_length = 25
tokenizer = Tokenizer(num_words=vocab_size, oov_token="OOV")
tokenizer.fit_on_texts(train_sentences)
train_sequences = tokenizer.texts_to_sequences(train_sentences)
padded_train_sequences = pad_sequences(train_sequences, maxlen = max_length, padding = 'post', truncating='post')
test_sequences = tokenizer.texts_to_sequences(test_sentences)
padded_test_sequences = pad_sequences(test_sequences, maxlen = max_length, padding = 'post', truncating = 'post')


In [19]:
train_sequences

[[320, 13336, 681, 3589, 2357, 46, 381, 2358, 13337, 6, 2750, 9270],
 [4, 7191, 2989, 2990, 22, 2, 154, 9271, 388, 2751, 6, 265, 9, 965],
 [156, 924, 2, 865, 1530, 2097, 599, 5049, 220, 135, 39, 45, 2, 9272],
 [1352, 37, 218, 382, 2, 1680, 29, 294, 22, 10, 2359, 1416, 5903, 1004],
 [715, 682, 5904, 1005, 9273, 662, 583, 5, 4, 95, 1292, 90],
 [9274, 4, 383, 71],
 [4, 7192, 372, 6, 470, 3590, 1979, 1467],
 [20, 567, 39, 1090, 31, 167, 2, 105, 84, 18, 146, 6, 33, 354],
 [269, 3591, 7193, 471, 7194, 1980, 147],
 [2215, 325, 350, 358, 62, 13338, 6, 4, 4462],
 [3275, 2098, 3950, 14, 36, 5050, 7195, 5, 2099, 966],
 [315, 756, 453, 8, 1598, 1778, 9, 3276],
 [226, 454, 2752, 13, 9, 925, 236, 373, 2, 3951, 13339, 7196],
 [233, 5051, 9275, 3277, 38, 223, 13340, 6, 7, 179],
 [13341, 1417, 757, 683, 5, 330, 3, 1006],
 [534, 2099, 9276, 128, 9277, 6, 9278, 3952, 1779],
 [2216,
  1353,
  344,
  48,
  3592,
  326,
  294,
  967,
  2,
  24,
  13342,
  19,
  1047,
  359,
  108,
  1354],
 [1599, 7197, 299

In [18]:
padded_train_sequences # distance giua cac tu

array([[  320, 13336,   681, ...,     0,     0,     0],
       [    4,  7191,  2989, ...,     0,     0,     0],
       [  156,   924,     2, ...,     0,     0,     0],
       ...,
       [ 1020,  3614,     5, ...,     0,     0,     0],
       [ 3702, 12639,    12, ...,     0,     0,     0],
       [ 1247,  1017,  1087, ...,     0,     0,     0]], dtype=int32)

In [20]:
tokenizer.index_word # chi tu word

{1: 'OOV',
 2: 'to',
 3: 'of',
 4: 'the',
 5: 'in',
 6: 'for',
 7: 'a',
 8: 'on',
 9: 'and',
 10: 'with',
 11: 'is',
 12: 'new',
 13: 'trump',
 14: 'man',
 15: 'from',
 16: 'at',
 17: 'about',
 18: 'you',
 19: 'by',
 20: 'this',
 21: 'after',
 22: 'up',
 23: 'out',
 24: 'be',
 25: 'how',
 26: 'that',
 27: 'it',
 28: 'as',
 29: 'not',
 30: 'are',
 31: 'your',
 32: 'what',
 33: 'his',
 34: 'all',
 35: 'he',
 36: 'who',
 37: 'just',
 38: 'has',
 39: 'will',
 40: 'more',
 41: 'into',
 42: 'one',
 43: 'year',
 44: 'report',
 45: 'have',
 46: 'over',
 47: 'area',
 48: 'why',
 49: 'donald',
 50: 'u',
 51: 'day',
 52: 'can',
 53: 'says',
 54: 's',
 55: 'first',
 56: 'woman',
 57: 'time',
 58: 'like',
 59: 'get',
 60: 'her',
 61: 'old',
 62: "trump's",
 63: 'no',
 64: 'now',
 65: 'off',
 66: 'an',
 67: 'life',
 68: 'obama',
 69: 'people',
 70: 'than',
 71: 'women',
 72: "'",
 73: 'house',
 74: 'back',
 75: 'was',
 76: 'still',
 77: 'white',
 78: 'make',
 79: 'down',
 80: 'clinton',
 81: 'my',
 

In [21]:
units = 128
embedding_size = 100
vocab_size = len(tokenizer.index_word)+1
input_length = max_length

In [22]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [23]:
rlrp = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2,patience=2)

In [24]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

In [26]:
model.fit(padded_train_sequences, train_label, batch_size = 32, epochs = 100, verbose = 1, callbacks = [rlrp], validation_data=(padded_test_sequences, test_label))

Epoch 1/100
668/668 [==============================] - 32s 41ms/step - loss: 0.3912 - acc: 0.8157 - val_loss: 0.3248 - val_acc: 0.8607 - lr: 0.0010
Epoch 2/100
668/668 [==============================] - 27s 40ms/step - loss: 0.1668 - acc: 0.9344 - val_loss: 0.3428 - val_acc: 0.8581 - lr: 0.0010
Epoch 3/100
668/668 [==============================] - 26s 38ms/step - loss: 0.0706 - acc: 0.9757 - val_loss: 0.4857 - val_acc: 0.8486 - lr: 0.0010
Epoch 4/100
668/668 [==============================] - 26s 39ms/step - loss: 0.0219 - acc: 0.9934 - val_loss: 0.6193 - val_acc: 0.8497 - lr: 2.0000e-04
Epoch 5/100
668/668 [==============================] - 26s 39ms/step - loss: 0.0095 - acc: 0.9978 - val_loss: 0.7477 - val_acc: 0.8491 - lr: 2.0000e-04
Epoch 6/100
668/668 [==============================] - 26s 38ms/step - loss: 0.0051 - acc: 0.9991 - val_loss: 0.7750 - val_acc: 0.8471 - lr: 4.0000e-05
Epoch 7/100
668/668 [==============================] - 26s 39ms/step - loss: 0.0045 - acc: 0.9992 - 

In [5]:
dataset

["former versace store clerk sues over secret 'black code' for minority shoppers",
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 "mom starting to fear son's web series closest thing she will have to grandchild",
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
 'j.k. rowling wishes snape happy birthday in the most magical way',
 "advancing the world's women",
 'the fascinating case for eating lab-grown meat',
 'this ceo will send your kids to school, if you work for his company',
 'top snake handler leaves sinking huckabee campaign',
 "friday's morning email: inside trump's presser for the ages",
 'airline passengers tackle man who rushes cockpit in bomb threat',
 'facebook reportedly working on healthcare features and apps',
 "north korea praises trump and urges us voters to reject 'dull hillary'",
 "actually, cnn's jeffrey lord has been 'indefensible' for a while",
 'barcelona holds huge protest in su

In [6]:
label_dataset

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,


In [ ]:
### Your Code Here
model = # TODO

In [ ]:
# Lưu Model và nộp
model.save("mymodel.h5")